<table>
    <tr><td align="right" style="background-color:#ffffff;">
        <img src="../images/logo.jpg" width="20%" align="right">
    </td></tr>
    <tr><td align="right" style="color:#777777;background-color:#ffffff;font-size:12px;">
        Prepared by Abuzer Yakaryilmaz<br>
        Özlem Salehi | August 04, 2019 (updated)
    </td></tr>
    <tr><td align="right" style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;">
        This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros.
    </td></tr>
</table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $

<h2>Rotation Automata</h2>

A decision problem is a problem which has the answer either yes or no. 

A rotation automaton solves a decision problem by making the decision of yes or no given a stream of symbols. The streams of symbols can be thought as instances of a problem. 

We focus on the streams composed by symbol $ a $s. So, the decisions will be basically about the lengths of streams.

We start with rotation automata having single qubits that are initialized to $ \ket{0} $.

We fix a rotation angle, and we apply this rotation for each symbol. 

After reading the stream, we make a measurement. If the outcome is '0', then we give one answer, and, if the outcome is '1', then we give another answer.  

Our aim is to make correct decisions on the streams as good as possible.

<h3> A trivial promise problem </h3>

Aim: Given a stream of $a$'s where the number of $a$'s is "promised" to be a multiple of $ 8 $, say yes if the length of the stream is an even multiple of $ 8 $. 

For each symbol $a$, we apply the rotation with angle $ \pi/16 $. In this way, we can exactly (zero-error) separate the streams having even multiples of $ 8 $ $a$'s from the streams having odd multiples of $ 8 $ $a$'s.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

# the angle of rotation
theta = pi/16

# we read streams of length 8, 16, 24, 32, 40, 48, 56, 64
for i in [8, 16, 24, 32, 40, 48, 56, 64]:
    # quantum circuit with one qubit and one bit
    qreg =  QuantumRegister(1) 
    creg = ClassicalRegister(1) 
    mycircuit = QuantumCircuit(qreg,creg)
    # the stream of length i
    for j in range(i):
        mycircuit.ry(2*theta,qreg[0]) # apply one rotation for each symbol
    # we measure after reading the whole stream
    mycircuit.measure(qreg[0],creg[0])
    # execute the circuit 100 times
    job = execute(mycircuit,Aer.get_backend('qasm_simulator'),shots=100)
    counts = job.result().get_counts(mycircuit)
   
    print("stream of lenght",i,"->",counts)
    print()
    


<b> Remark:</b> For the same problem, we need at least 4 classical bits. 

When changing the parameter $2^3$ to $ 2^k $, we can still use a single qubit. On the other hand, we need at least $ (k+1) $ classical bits.

<h3>Task 1</h3>

Do the same task given above by using different angles.

Test at least three different angles. 

Please modify the code above.

<a href="B72_Rotation_Automata_Solutions.ipynb#task1">click for our solution</a>

<h3>$ \mathsf{MOD_p} $</h3>

Now, we consider a more general problem called $ \mathsf{MOD_p} $, where $\sf p$ is a prime number.

We will read a stream of symbols $a$. The number of $a$'s can be <b>arbitrary</b>.

For each symbol, we apply a rotation.

Our aim is to separate the streams whose lengths are multiple of $ \sf p $ from the other streams. 

<b>We design a good decider step by step.</b>

<i>Remark that each $ p $ defines a different problem.</i>

<h3>Task 2</h3>

Let $ \mathsf{p} = 11 $.

Determine an angle of rotation such that when the length of stream is a multiple of $ \sf p $, then we observe only state $ 0 $, and we can also observe state $ 1 $, otherwise.

Test your rotation by using a quantum circuit. Execute the circuit for all streams of lengths from 1 to 11.

In [ ]:
#
# your solution is here
#

# the streams of lengths from 1 to 11
for i in range(1,12):
    

<a href="B72_Rotation_Automata_Solutions.ipynb#task2">click for our solution</a>

<h3>Task 3</h3>

List down 10 possible different angles for Task 2, where each angle should be between 0 and $2\pi$.

<a href="B72_Rotation_Automata_Solutions.ipynb#task3">click for our solution</a>

<h3> Observation</h3>

We have seen that we can choose different multiples of $\frac{2\pi}{11}$ and in each case we only observe state $\ket{0}$ when the string's length is a multiple of $p$. 

For some streams of lengths from 1 to 10, we observe state $\ket{1}$ only for a few times but observe $\ket{0}$ rest of the time.

This is definitely not desirable if we wish to distinguish the lengths that are multiple of $\sf p$ from the rest with high probability.

<h3>Task 4</h3>

Let $p=11$.

Now suppose that we are trying to distinguish two strings which have length $n$ and $p$ from each other.

For each different $n$, there is a different angle (which is a multiple of $\frac{2\pi}{11}$ which works better, that is the one which you observe state $\ket{1}$ the most.
 
Experimentially determine the best angle of rotation for each $n$ by using your circuit.

In [ ]:
#
# your solution is here
#


<a href="B72_Rotation_Automata_Solutions.ipynb#task4">click for our solution</a>

<h3> Random strategy for $ \mathsf{MOD_p} $</h3>

For different length of streams that are not multiple of $\sf p$, different angles $ \big( k\frac{2\pi}{p} \big) $ work better.

We can use more than one rotation automaton, each of which uses different $ k $ for its angle of rotation. 

In this way, we can get better results, i.e., we can distinguish the lengths not multiple of $\sf p$ with better probabilities, i.e., we observe state $ \ket{1} $ more than half times for example.

<h3>Task 5</h3>

Let $ \mathsf{p} = 31 $.

Create a circuit with three quantum bits and three classical bits.

Rotate the qubits with angles $ 3\cdot \frac{2\pi}{31} $, $ 7\cdot\frac{2\pi}{31} $, and $ 11\cdot\frac{2\pi}{31} $, respectively.

Execute your circuit for all streams of lengths from 1 to 30. Check whether the number of state $ \ket{000} $ is less than half or not.

<i>Note that whether a key is in dictionary or not can be checked as follows:</i>

    if '000' in counts.keys():
        c = counts['000']
    else:
        c = 0

In [ ]:
#
# your solution is here
#

# the streams of lengths from 1 to 30
for i in range(1,31):
    

<a href="B72_Rotation_Automata_Solutions.ipynb#task5">click for our solution</a>

<h3>Task 6</h3>

Let $ \mathsf{p} = 31 $.

Create a circuit with three quantum bits and three classical bits.

Rotate the qubits with random angles of the form $ k\frac{2\pi}{31} $, where $ k 
\in \{1,\ldots,30\}.$

Execute your circuit for all streams of lengths from 1 to 30.

Calculate the maximum percentage of observing the state $ \ket{000} $.

Repeat this task for a few times.

<i>Note that whether a key is in dictionary or not can be checked as follows:</i>

    if '000' in counts.keys():
        c = counts['000']
    else:
        c = 0

In [ ]:
#
# your solution is here
#

# the streams of lengths from 1 to 30
for i in range(1,31):
    

<a href="B72_Rotation_Automata_Solutions.ipynb#task6">click for our solution</a>

<h3>Task 7</h3>

Repeat Task 6 by using four and five qubits.

In [ ]:
#
# your solution is here
#

# the streams of lengths from 1 to 30
for i in range(1,31):
    

<a href="B72_Rotation_Automata_Solutions.ipynb#task7">click for our solution</a>

<h3>Remarks</h3>

Problem $\sf MOD_p$ can be classically solved by using no less than $\sf p$ states.

As we have observed, the same problem can be solved by using a few quantum states in some cases.

In fact, the above given random strategy can be implemented more state efficiently (we discuss the basics of this technique in the next notebook) so that $ \log \mathsf{p} $ quantum states is enough to solve the problem with high probability.

Thus, we need at least logarithmic number of classical bits. On the other hand, we can use double logarithmic quantum bits. This is another exponential advantage of quantum computation.  

<i> One implementation issue for the quantum algorithm is to implement more and more precise rotations, i.e., implementing the rotation with angle $ \frac{2\pi}{p} $ may not be possible when $ p $ gets bigger and bigger.

Besides, a long sequence of rotations may require some error corrections, and each error correction solution can use new qubits.
</i>